In [1]:
from importlib import reload
import LambdaRankHW
import numpy as np
import query
import os
import matplotlib.pyplot as plt
import pandas as pd

DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/Dana/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.5.2-64/tmp4rhcfr62/m91973e5c136ea49268a916ff971b7377.lib and object C:/Users/Dana/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.5.2-64/tmp4rhcfr62/m91973e5c136ea49268a916ff971b7377.exp

Using gpu device 0: GeForce 940MX (CNMeM is enabled with initial size: 80.0% of memory, CuDNN 5110)
C:\Users\Dana\Anaconda3\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)
C:\Users\Dana\Anaconda3\lib\site-packages\theano\tensor\signal\do

In [ ]:
train_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/train.txt'), 64)
test_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/test.txt'), 64)
val_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/vali.txt'), 64)

In [ ]:
%%prun
reload(LambdaRankHW)
n_epochs = 100
measure_type = LambdaRankHW.LISTWISE
num_features = 64
num_folds = 1

res = LambdaRankHW.experiment(n_epochs, measure_type, num_features, num_folds)


Loading train queries


In [ ]:
valndcgs = [[rr['val_mndcg'] for rr in res[i][:-1]] for i in res]  
trainndcgs = [[rr['train_mndcg'] for rr in res[i][:-1]] for i in res]  

df = pd.DataFrame(valndcgs, index=['Val Fold%d' % d for d in range(1, num_folds + 1)]).T
plt.plot(df)
df2 = pd.DataFrame(trainndcgs, index=['Train Fold%d' % d for d in range(1, num_folds + 1)]).T
plt.plot(df2)
plt.legend(df2.columns + df.columns)
plt.show()

In [ ]:
df